In [4]:
import os
import psycopg2
import pandas as pd
import json, ast

from importlib import reload

from dotenv import load_dotenv

from pgvector.psycopg2 import register_vector

from utils import api, db, prompt, prompting

load_dotenv()

conn = psycopg2.connect(os.getenv("POSTGRES_CONFIG"))
register_vector(conn)
cursor = conn.cursor()

In [5]:
reload(prompt)
reload(api)
reload(db)
reload(prompting)

<module 'utils.prompting' from '/Volumes/Personal/Work/Feedloop/fact_ext/from_github/AI-Research/utils/prompting.py'>

# Trial

In [56]:
df = pd.read_csv('./test_set/BPK.csv')

In [ ]:
for index, row in df.iterrows():
    question = row['question']
    expected_answer = row['ground_truth']

In [57]:
pdf_path = "./pdfs/Test BPK - summary_fact.pdf"

In [58]:
resource_id = db.get_resource_id(cursor, "Test BPK - summary_fact")
print(resource_id)

27682ca8-2a2c-4580-a776-e17dd2843a45


In [82]:
query = df['question'][2]
resource_ids = [resource_id]
knowledges = db.get_retrieved_knowledge(cursor, query, resource_ids, 10)
retrieved = [k[1] for k in knowledges]

In [83]:
chat_prompt = prompt.get_chat_prompt(query, retrieved, memory_max_tokens=500, config="context")
print(chat_prompt)

[BOT]
--- SYSTEM INFO:
Today's Time: Friday, 05 January 2024 07:53AM
--- BOT INSTRUCTION

--- USER INFO:
{"name":"John","email":"john@feedloop.ai"}

--- KNOWLEDGE:
1. Test BPK - summary_fact.pdf
Doc URL: None603f9138-2723-4b28-bc56-c32190521e64/Test BPK - summary_fact.pdf
Context 1: Badan Pemeriksa Keuangan Republik Indonesia (BPK RI) memiliki tugas dan wewenang yang dibagi antara Ketua, Wakil Ketua, dan Anggota BPK. Tugas dan wewenang Ketua BPK meliputi pemeriksaan pengelolaan keuangan negara, kelembagaan BPK, hubungan kelembagaan dalam dan luar negeri, serta pembinaan tugas beberapa direktorat dan badan. Wakil Ketua BPK bertanggung jawab atas proses Majelis Tuntutan Perbendaharaan dan memberikan pengarahan pemeriksaan investigatif. Sedangkan Anggota BPK bertugas melakukan pemeriksaan pengelolaan keuangan negara.
Facts from Context 1:
- B. Tugas dan Wewenang Pimpinan Badan Pemeriksa Keuangan Republik Indonesia (BPK RI) saat ini adalah sebagai berikut:
- b. Tugas dan wewenang Wakil Ket

In [84]:
completion = api.get_completions_gpt4([
        {
        "role": "user",
        "content": chat_prompt,
        }
    ], temp=0)

In [86]:
gpt_answer = json.loads(completion.choices[0].message.content)['response'][0]['text']
print(gpt_answer)

Maaf, saya tidak memiliki informasi tentang siapa Wakil Ketua BPK RI saat ini. Namun, berdasarkan dokumen yang saya miliki, tugas dan wewenang Wakil Ketua BPK RI adalah sebagai berikut:
1. Melaksanakan pemeriksaan pengelolaan dan tanggung jawab keuangan negara secara umum bersama dengan Ketua.
2. Melaksanakan proses Majelis Tuntutan Perbendaharaan.
3. Melaksanakan pembinaan tugas Sekretariat Jenderal, Direktorat Utama Perencanaan, Evaluasi, dan Pengembangan Pemeriksaan Keuangan Negara, Direktorat Utama Pembinaan dan Pengembangan Hukum Pemeriksaan Keuangan Negara, Inspektorat Utama, dan Badan Pendidikan dan Pelatihan Pemeriksaan Keuangan Negara bersama dengan Ketua.
4. Memberikan pengarahan pemeriksaan investigatif.


In [87]:
expected_answer = df['ground_truth'][2]
eval_prompt = prompt.get_eval_prompt(query, expected_answer, gpt_answer)
print(eval_prompt)

Question:
Siapa Wakil Ketua BPK RI saat ini dan apa tugas serta wewenangnya dalam pemeriksaan keuangan negara?
---

Expected answer:
Wakil Ketua BPK RI saat ini adalah Dr. Agus Joko Pramono, M.Acc., Ak., CA., CSFA., CPA., CFrA., QGIA., CGCAE. Tugas dan wewenangnya dalam pemeriksaan keuangan negara adalah melaksanakan pemeriksaan pengelolaan dan tanggung jawab keuangan negara secara umum bersama dengan Ketua BPK.
---

Current answer:
Maaf, saya tidak memiliki informasi tentang siapa Wakil Ketua BPK RI saat ini. Namun, berdasarkan dokumen yang saya miliki, tugas dan wewenang Wakil Ketua BPK RI adalah sebagai berikut:
1. Melaksanakan pemeriksaan pengelolaan dan tanggung jawab keuangan negara secara umum bersama dengan Ketua.
2. Melaksanakan proses Majelis Tuntutan Perbendaharaan.
3. Melaksanakan pembinaan tugas Sekretariat Jenderal, Direktorat Utama Perencanaan, Evaluasi, dan Pengembangan Pemeriksaan Keuangan Negara, Direktorat Utama Pembinaan dan Pengembangan Hukum Pemeriksaan Keuangan N

In [88]:
eval_completion = api.get_completions_gpt4([
        {
        "role": "user",
        "content": eval_prompt,
        }
    ], temp=0)

In [89]:
print(eval_completion.choices[0].message.content)

{ 
    "similarity": 60,

    "correctness": 50,

    "relevancy": 70,

    "overall": 60,

    "reasons": "The current answer does not provide the name of the current Vice Chairman of BPK RI, which is part of the question. However, it does provide a detailed explanation of the duties and authorities of the Vice Chairman in the financial audit of the state, which is relevant and partially correct.",

}


In [90]:
json_path = './gpt_answer/Fact New.json'
with open(json_path, 'r') as json_file:
    json_answer_data = json.load(json_file)

In [91]:
print(json_answer_data['Coffee_and_health'][0])

{'question': 'What health benefits does coffee consumption offer and how do they differ based on individual sensitivity and genetic polymorphisms?', 'answer': "Moderate coffee consumption is safe for health and can even provide health benefits. It has positive health effects on various aspects of human health. Coffee can help prevent some chronic diseases such as type 2 diabetes, liver damage, and Parkinson's disease. It contains antioxidant compounds, especially chlorogenic acid which is a strong antioxidant. However, the caffeine content in coffee can be harmful to pregnant women and children if consumed excessively. Coffee can affect the absorption of calcium in the digestive tract and can increase the risk of osteoporosis. The diterpene content in coffee can increase cholesterol, but most diterpenes are removed by paper filters. Coffee can have cardiovascular effects such as tachycardia, high blood pressure, and arrhythmia. Coffee consumption can reduce the risk of suicide by 13% e

# Implementation

In [199]:
total_eval_prompt_token = 0
total_eval_completion_token = 0

In [200]:
def run_eval(json_path, doc_name, csv_path):
    global total_eval_prompt_token, total_eval_completion_token
    with open(json_path, 'r') as json_file:
        json_answer_data = json.load(json_file)
    
    cur_doc_answer = json_answer_data[doc_name]
    question_list, answer_list, truth_list = [], [], []
    for i in cur_doc_answer:
        question_list.append(i['question'])
        answer_list.append(i['answer'])
        ground_truth = ""
        for j in i['ground_truths']:
            ground_truth+=f'{j}\n'

        truth_list.append(ground_truth)

    json_eval_result = []
    for i in range(len(question_list)):
        eval_prompt = prompt.get_eval_prompt(question_list[i], truth_list[i], answer_list[i])
        eval_completion = api.get_completions_gpt4([
            {
            "role": "user",
            "content": eval_prompt,
            }
        ], temp=0)
        total_eval_prompt_token+=eval_completion.usage.prompt_tokens
        total_eval_completion_token+=eval_completion.usage.completion_tokens
        json_eval_result.append(json.loads(eval_completion.choices[0].message.content))
    
    df_eval_result = pd.DataFrame(json_eval_result)
    df_eval_result.to_csv(csv_path)

In [253]:
json_path = "./gpt_answer/mul_SentWindow.json"
csv_path = "./eval_result/mul_SentWindow/Paper.csv"
doc_name = "Coffee_and_health"
run_eval(json_path, doc_name, csv_path)

In [214]:
print(f"eval prompt token: {total_eval_prompt_token}\neval completion token: {total_eval_completion_token}")
eval_prompt_token_price = total_eval_prompt_token * 0.01 * 0.001
eval_completion_token_price = total_eval_completion_token * 0.03 * 0.001
print(f"eval prompt token price: {round(eval_prompt_token_price,2)}\neval completion token price: {round(eval_completion_token_price,2)}")

eval prompt token: 34396
eval completion token: 7438
eval prompt token price: 0.34
eval completion token price: 0.22


# Eval from Test_Set

In [215]:
total_prompt_token = 0
total_completion_token = 0

In [228]:
df_test_set = pd.read_csv('./test_set/Paper.csv')

In [7]:
# pdf_path = './pdfs/Perbup Sleman Nomor 11.3 Tahun 2019 ttg Pengelolaan keuangan desa - context_fact.pdf'
# resource_ids = db.get_resource_ids(cursor, "Test BPK -%")

In [243]:
extract_method = " - SentWindow"
doc_list = []
doc_list.append("CS Knowledges"+extract_method)
doc_list.append("Coffee_and_health"+extract_method)
doc_list.append("Test BPK"+extract_method)
doc_list.append("FL Healthcare Use Case - Medical Record"+extract_method)
doc_list.append("General Knowledge"+extract_method)
doc_list.append("KORAN-TEMPO"+extract_method)
doc_list.append("Perbup Sleman Nomor 11.3 Tahun 2019 ttg Pengelolaan keuangan desa"+extract_method)

In [244]:
resource_ids = []
for doc in doc_list:
    resource_ids.append(db.get_resource_id(cursor, doc))

In [245]:
print(len(resource_ids))

7


In [247]:
test_answer = []
for _, row in df_test_set.iterrows():
    question = row["question"]
    retrieved = db.get_retrieved_knowledge(cursor, question, resource_ids, 25)
    retrieved = [k[1] for k in retrieved]

    config = "no_overlap"
    lang = "english"
    chat_prompt = prompt.get_chat_prompt(question, retrieved, memory_max_tokens=500, config=config, lang=lang)
    knowledges = prompt.get_knowledge_from_prompt(chat_prompt)
    gpt4_completion = api.get_completions_gpt4([
        {
            "role": "user",
            "content": chat_prompt,
        }
    ], temp=0)

    total_prompt_token+=gpt4_completion.usage.prompt_tokens
    total_completion_token+=gpt4_completion.usage.completion_tokens
    
    response = gpt4_completion.choices[0].message.content
    try:
        response = ast.literal_eval(response)
    except:
        response = ast.literal_eval(response.replace("null", "None"))

    if isinstance(response["response"], list):
        answer = response["response"][0]["text"]
    elif isinstance(response["response"], dict):
        answer = response["response"]["text"]
    else:
        ValueError(f"{response}")

    test_answer.append({
        "question": question,
        "answer": answer,
        "contexts": knowledges,
        "ground_truths": [row["ground_truth"]]
    })

In [248]:
# dict_answer = {}
with open('./gpt_answer/mul_SentWindow.json', 'r') as json_file:
    dict_answer = json.load(json_file)

In [249]:
dict_answer["Coffee_and_health"] = test_answer

In [250]:
with open('./gpt_answer/mul_SentWindow1.json', 'w') as json_file:
    json.dump(dict_answer, json_file, indent=2)

In [193]:
print(f"prompt token: {total_prompt_token}\ncompletion token: {total_completion_token}")

prompt token: 114764
completion token: 28188


In [194]:
prompt_token_price = total_prompt_token * 0.01 * 0.001
completion_token_price = total_completion_token * 0.03 * 0.001
print(f"prompt token price: {prompt_token_price}\ncompletion token price: {completion_token_price}")

prompt token price: 1.1476400000000002
completion token price: 0.8456400000000001
